## Convolutional and Max Pooling Layers

In order to get a better understanding of convolutional and max pooling layers, in this homework you will implement these two layers from scratch without PyTorch! Fill out the two python functions below and then run the `assert` statements in order to check that your code works :)

Homework idea based off of assignment 2 from CS231N

In this homework, you are only allowed to use the `numpy` package. I think this homework is fairly hard so please feel free to discuss with classmates on piazza. However, please post at most pseudocode for a subproblem and please do not post an answer.  

In [17]:
import numpy as np

![ConvURL](https://raw.githubusercontent.com/iamaaditya/iamaaditya.github.io/master/images/conv_arithmetic/full_padding_no_strides_transposed.gif "conv")

Recall the convolution layer. In this gif, we slide a 3 x 3 (dark blue) filter along the input image (light blue) in order to produce the (green) output image. There is no padding, and the stride is 1. To produce a green value, we dot a portion of the input image with the filter weights and add the bias ($Wx + b$).

### 1. Convolutional Layer

Implement `conv_forward_naive`, which takes in the input data, the weight matrix, the bias vector, and parameters about this convolutional layer.

Hint: It may be a good idea to extract out N, C, H, W, F, etc. into variables for easier use.

Hint2: How many `for` loops do you need?


In [18]:
def conv_forward_naive(x, w, b, conv_param):
    """
    A naive implementation of the forward pass for a convolutional layer.

    The input consists of N data points, each with C channels, height H and
    width W. We convolve each input with F different filters, where each filter
    spans all C channels and has height HH and width WW.

    Input:
    - x: Input data of shape (N, C, H, W)
    - w: Filter weights of shape (F, C, HH, WW)
    - b: Biases, of shape (F,)
    - conv_param: A dictionary with the following keys:
      - 'stride': The number of pixels between adjacent receptive fields in the
        horizontal and vertical directions.
      - 'pad': The number of pixels that will be used to zero-pad the input.

    Returns:
    - out: Output data, of shape (N, F, H', W') where H' and W' are given by
      H' = 1 + (H + 2 * pad - HH) / stride
      W' = 1 + (W + 2 * pad - WW) / stride
    """
    out = None
    
    stride = conv_param['stride']
    padding = conv_param['pad']

    n = x.shape[0]
    c = x.shape[1]
    h = x.shape[2]
    width = x.shape[3]
    
    f = w.shape[0]
    hh = w.shape[2]
    ww = w.shape[3]

    h_prime = int(1 + (h + 2 * padding - hh) / stride)
    w_prime = int(1 + (width + 2 * padding - ww) / stride)

    npad = ((0, 0), (0, 0), (padding, padding), (padding, padding))
    x_pad = np.pad(x, pad_width=npad, mode='constant', constant_values=0)

    out = np.zeros((n, f, h_prime, w_prime))
    n_pad, c_pad, h_pad, w_pad = x_pad.shape
   

    ###########################################################################
    # TODO: Implement the convolutional forward pass.                         #
    # Hint: you can use the function np.pad for padding. We did this for you. #
    # We also defined all the salient variables. The remaining part of this   #
    # problem is creating the for loops and stitching the variables together. #
    # Start by making sure you understand every portion of the above code.    #
    # Hint2: Add the appropriate bias to each term in each convolution        #
    ###########################################################################
    
    for N in range(n):
        for F in range(f):
            for i in range(0, h_pad - (hh - 1), stride):
                for j in range(0, w_pad - (ww - 1), stride):
                    image_to_be_weighted = x_pad[N, :, i:i+hh, j:j+ww]
                    weights = w[F,...]
                    product = np.sum(np.multiply(weights,image_to_be_weighted))
                    row_block = int(i/stride)
                    col_block = int(j/stride)
                    out[N, F, row_block, col_block] = product + b[F]
    print("out is ", out)


    
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return out

### 2. Max Pooling Layer

Implement this max pooling layer python function.

Hint: This should be pretty similar to the convolution layer above.

Hint2: It will be useful to calculate what the expected output dimensions will be. If you need help with this, feel free to chat a friend in the DeCal or a staff member.

In [19]:
def max_pool_forward_naive(x, pool_param):
    """
    A naive implementation of the forward pass for a max pooling layer.

    Inputs:
    - x: Input data, of shape (N, C, H, W)
    - pool_param: dictionary with the following keys:
      - 'pool_height': The height of each pooling region
      - 'pool_width': The width of each pooling region
      - 'stride': The distance between adjacent pooling regions

    Returns:
    - out: Output data
    """
    out = None
    ###########################################################################
    # TODO: Implement the max pooling forward pass                            #
    ###########################################################################
   
    height_pool = pool_param['pool_height']
    width_pool = pool_param['pool_width']
    stride = pool_param['stride']
    n, c, h, w = x.shape 
    h1 = (h - height_pool) // stride + 1
    w1 = (w - width_pool) // stride + 1
    out = np.zeros((n, c, h1, w1))
    for i in range(n):
        for j in range(c):
            for k in range(h1):
                for l in range(w1):
                    out[i,j,k,l] = np.max(x[i, j, k * stride:k * stride + height_pool, 1 * stride:1 * stride + width_pool])

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    print(out)
    return out

## 3. Check your answers!

If your code passes these assert statements, you should be good to go!

In [20]:
### TESTING OF CONV LAYER

np.random.seed(42)
x = np.random.normal(size=[1, 3, 11, 11])
w = np.random.normal(size=[2, 3, 5, 5])
b = np.random.normal(size=[2])
conv_param = {'stride': 3, 'pad': 3}
result = conv_forward_naive(x, w, b, conv_param)
assert np.allclose(result, np.array([[[[ 3.13783023, -5.32751231, -1.71460357, -3.22003339,  3.27438643],
   [-3.39967189,  2.39404469, -4.2126656,   4.80549383, -1.40836569],
   [-0.67034629, -7.53964901, -8.11099708, -6.24694429, -1.82490217],
   [ 0.75443863, -4.92723594,  3.06248213, -2.37856105,  8.86919592],
   [-7.34305417,  3.18673458,  6.33894349, -1.72720915,  0.77468496]],

  [[-2.61841618,  3.32434937, -0.93731549,  3.27477707,  0.63882942],
   [-6.49573417, -0.33254641,  0.93942528, 15.50203272, -3.9097889 ],
   [ 6.75564931, -3.56840551,  4.27588487, 12.28841061, -6.50030743],
   [ 0.59184847,  2.48632324,  4.99003361,  5.70073028, -2.3884948 ],
   [-3.98443104,  1.29550344, -3.46922113, -2.73797865, -1.05677199]]]]))

out is  [[[[ 3.13783023 -3.39967189 -0.67034629  0.75443863 -7.34305417]
   [-5.32751231  2.39404469 -7.53964901 -4.92723594  3.18673458]
   [-1.71460357 -4.2126656  -8.11099708  3.06248213  6.33894349]
   [-3.22003339  4.80549383 -6.24694429 -2.37856105 -1.72720915]
   [ 3.27438643 -1.40836569 -1.82490217  8.86919592  0.77468496]]

  [[-2.61841618 -6.49573417  6.75564931  0.59184847 -3.98443104]
   [ 3.32434937 -0.33254641 -3.56840551  2.48632324  1.29550344]
   [-0.93731549  0.93942528  4.27588487  4.99003361 -3.46922113]
   [ 3.27477707 15.50203272 12.28841061  5.70073028 -2.73797865]
   [ 0.63882942 -3.9097889  -6.50030743 -2.3884948  -1.05677199]]]]


AssertionError: 

In [21]:
### TESTING OF MAX POOL LAYER
np.random.seed(45)
x = np.random.normal(size=(1, 3, 8, 8))
pool_param = {'pool_height': 5, 'pool_width': 5, 'stride': 3}
result = max_pool_forward_naive(x, pool_param)

assert np.allclose(result, np.array([[[[ 2.24808957,  2.24808957],
         [ 2.24808957,  2.24808957]],

        [[ 1.21650079,  1.81659525],
         [ 2.44659327,  1.76020474]],

        [[ 2.99363398,  1.62741182],
         [ 1.67882964,  1.67882964]]]]) )

[[[[2.24808957 2.24808957]
   [2.24808957 2.24808957]]

  [[1.81659525 1.81659525]
   [1.76020474 1.76020474]]

  [[1.62741182 1.62741182]
   [1.67882964 1.67882964]]]]


AssertionError: 